# VERTEX AI


In [25]:
# pip install --upgrade google-cloud-aiplatform
# gcloud auth application-default login

In [11]:
#!gcloud auth application-default login

^C


In [2]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, ChatSession
import vertexai.preview.generative_models as generative_models
import pprint

In [3]:
from dotenv import dotenv_values

config = dotenv_values(".env")

In [4]:
vertexai.init(project=config['PROJECT'], location=config['REGION'])

In [5]:
def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        text_response.append(chunk.text)
    return "".join(text_response)

In [62]:
model = GenerativeModel(
    "gemini-1.5-pro-preview-0409",
    system_instruction=["""You a helpful agent who helps to extract relevant information from documents"""],
    safety_settings= {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
},
     generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.1,
    "top_p": 0.95,
}
  )
chat = model.start_chat(response_validation=False)

In [63]:
from pathlib import Path
content=[]
pathlist = Path("data/tasaciones").glob('**/*.pdf')
for path in pathlist:
    # Convert path object to a string
    path_in_str = str(path)
    # Perform actions on the file (e.g., print its path)
    print(path_in_str)
    with open(path_in_str, "rb") as f:
        im_bytes = f.read()
        im_b64 = base64.b64encode(im_bytes).decode("utf8")
        image = Part.from_data(data=im_b64, mime_type="application/pdf")
        content.append(image)

data\tasaciones\F06MP58U8.pdf


In [64]:
question = """Instrucciones:
              1) Extrae de este documento la descripcion completa del inmueble.
              2) registrador.
              3) solicitante del documento.
              4) fecha documento.
              5) La respuesta debe ser en el lenguage del documento.

"""

In [65]:
prompt =[f"""{question} """]+  content

In [66]:
response = get_chat_response(chat, prompt)

In [67]:
print(response)

## Información extraída del documento:

**1) Descripción completa del inmueble:**

URBANA.- PARCELA NUMERO 183. Sita en el término municipal de Pelabravo (Salamanca) núcleo de Naharros, en el sitio denominado "Cuesta Chica". Tiene una SUPERFICIE de trescientos cincuenta y un (351,00) metros cuadrados. Tiene entrada por la Calle Cuarta. LINDA: Norte, Calle Cuarta; Sur, Parcela número 184; Este, Parcela número 185; Oeste: Parcela número 181. Residencial-Vivienda Unifamiliar. Carácter Privado. Tiene asignado un coeficiente de 0'417828%, quedando afectada por un importe de 12.880,01 euros a cuenta de la liquidación provisional y definitiva para gastos de urbanización.

**2) Registrador:**

JOAQUIN JOSE RODRIGUEZ HERNANDEZ
Registrador de la Propiedad de SALAMANCA 2

**3) Solicitante del documento:**

TECNOTRAMIT SERVICIOS DE GESTION SL.
con DNI/CIF: B62593363

**4) Fecha del documento:**

VEINTINUEVE DE MARZO DEL AÑO DOS MIL DIECINUEVE 



In [68]:
question = """Extrae Titulares actuales y su DNI o CIF"""
prompt=  [f"""{question} """]


In [69]:
response1 = get_chat_response(chat, prompt)
        

In [70]:
print(response1)

## Titulares actuales y su DNI o CIF:

* **SOCIEDAD DE GESTION DE ACTIVOS PROCEDENTES DE LA REESTRUCTURACION BANCARIA SA**
* **DNI/CIF:** A-86602158 



In [71]:
question = """Tiene cargas la vivienda? si es puedes enumerarlas"""
prompt=  [f"""{question} """]

In [72]:
response2 = get_chat_response(chat, prompt)
print(response2)

## Cargas de la vivienda:

Sí, la vivienda tiene una carga:

* **AFECCIÓN URBANÍSTICA:** Afecta con carácter real al pago del saldo de la liquidación definitiva de la cuenta del Proyecto de Actuación en que se generó, en proporción a la cuota a ella asignada. 
* **Detalles de la afección:** Certificación administrativa de fecha 05/03/07 del AYUNTAMIENTO PELABRAVO. Inscripción 1ª, al tomo 3.786, libro 68, folio 148, de fecha 10/07/2007. 



In [73]:
question = """Extrae los Datos registrales de la finca"""
prompt=  [f"""{question} """]

In [74]:
response3 = get_chat_response(chat, prompt)
print(response3)

## Datos registrales de la finca:

* **CÓDIGO REGISTRAL ÚNICO de finca:** 37010001056191
* **FINCA DE PELABRAVO N°:** 5365 



In [19]:
chat.history[0]

role: "user"
parts {
  text: "Instrucciones:\n              1) Extrae de este documento la descripcion completa del inmueble.\n              2) registrador\n              3) solicitante\n              4) fecha documento\n\n "
}
parts {
  inline_data {
    mime_type: "application/pdf"
    data: "%PDF-1.4\r%\342\343\317\323\r1 0 obj\n<<\n/Type/ExtGState\n/SA false\n/SM 0.02\n>>\nendobj\n2 0 obj\n[/DeviceRGB]\nendobj\n3 0 obj\n<</Filter/FlateDecode/Length 1571>>\nstream\nx\332\235X\315\216\0337\014\276\373)\346\005<+Q\242~\200b\000\2577.\332[\320\005z(rJ\233\236\332\"\315\373\003\245,\222\222F\263N\032,\274\2269$\365\361G\322\247y\372\361\027\273\374\371\345\364t\245\357\217_NfEc\027C\177_>\376}z~==\335\334b\315\362\372\351d\301\255y\211\036\326\270\274\376~\372\355\007c\214\333>,\257?\237\336\275\312S\227\327\214\213\313n1\253_\376\375\343\364\351\364\371t\377I\177\301\254\026\227\336\317\307\277NO?\331\345\345\237\323{\3223K\262nu\270`\306\362\005\251\374\'\'\305\230e\021\342j\213\354\3

In [20]:
chat.history[1]

role: "model"
parts {
  text: "## Información extraída del documento:\n\n**1) Descripción completa del inmueble:**\n\n* **Tipo:** Vivienda en la planta segunda puerta primera.\n* **Ubicación:** Calle Gandesa número tres, Barcelona.\n* **Superficie:** 85.60 metros cuadrados.\n* **Lindes:**\n    * Frente: Rellano de escalera y vivienda puerta segunda.\n    * Derecha: Patio y vivienda puerta cuarta.\n    * Izquierda: Patio y casa número de la calle Gandesa.\n    * Fondo: Casa número 183 de la calle Europa y patio.\n* **Coeficiente en la propiedad horizontal:** 2.88%\n* **Referencia catastral:** 7323304DF2872C0007ME\n\n**2) Registrador:**\n\n* Registrador de la Propiedad de BARCELONA N° 7\n* Paseo de la Zona Franca, número 109\n* 08038- Barcelona\n\n**3) Solicitante:**\n\n* ALBERT MORGADES ABELLA\n* DNI/CIF: 77787768C\n\n**4) Fecha del documento:**\n\n* 3/10/2019 \n"
}

In [21]:
chat.history[2]

role: "user"
parts {
  text: "Extrae Titulares actuales y su DNI o CIF "
}

In [22]:
chat.history[3]

role: "model"
parts {
  text: "## Titulares actuales y su DNI/CIF:\n\n* **Nombre:** don RAÚL RODRÍGUEZ SASO\n* **DNI/CIF:** N.I.F. 46139534Q \n"
}

In [23]:
chat.history[4]

role: "user"
parts {
  text: "Tiene cargas la vivienda? si es puedes enumerarlas "
}

In [24]:
chat.history[5]

role: "model"
parts {
  text: "## Cargas de la vivienda:\n\nSegún el documento, la vivienda está **libre de cargas**. \n"
}

In [25]:
chat.history[6]

role: "user"
parts {
  text: "Extrae los Datos registrales de la finca "
}

In [26]:
chat.history[7]

role: "model"
parts {
  text: "## Datos registrales de la finca:\n\n* **Código único de Finca:** 08060000358435\n* **Tomo:** 1394\n* **Libro:** 1394\n* **Folio:** 178 \n"
}